# "`matchingmarkets`" simulation package tutorial (part 1)

This package simulates matching markets (without money). The simulated markets can be for a single period (static) or evolve over multiple periods (dynamic). 

For example, a single period matching market could be matching high school students to schools, which is done at once for all high school freshmen each year. In contrast, a dynamic market could be an organ transplant list, matching patients to organs. In the transplant list, patients and organs arrive in the market at different times, and leave the market at different times (either by receiving a transplant, or death).

It's intuitive that methods built for static markets won't necessarily perform well in dynamic markets, because the decision of *when* to match is now important.

While the problem of static matching markets has been well studied over many decades, the study of dynamic matching markets is recent and sparse in comparison. For instance, [Akbarpour et al. (2014)](http://economics.mit.edu/files/10375) is the first known publication to study the theory of dynamic matching in general matching markets. 

It's reasonable to assume that the lack of formal theory around dynamic matching is because of the sheer mathematical difficulty of proving results in this domain, and not because of lack of applicability, or interest (recall the 2012 [Nobel prize in economics](http://www.nobelprize.org/nobel_prizes/economic-sciences/laureates/2012/) was given to Shapley and Roth for their study of matching markets.)

This lack of formal theory motivates this package, which is built to test user defined algorithms in a fairly general setting, with a focus on dynamic markets. The `matchingmarkets` package is built around the following concepts: various types of **Agents**, a **Market** containing the agents, **Algorithms** which decide *who* to match in the market and *how*, and **meta-Algorithms** which decide *when*, and with *who*, to implement the algorithm. 

In this tutorial, we'll explain these concepts, then use them to test the theoretical results in Akbarpour et al. (2014) using the package.

# Agents

Agents are the relevant members in a matching market. In the `matchingmarkets` package they are represented by the `Agent` class, which has a few attributes to allow it to represent general agents in a market. An agent is initialized in python with

In [3]:
import matchingmarkets as mm

mrAgent = mm.Agent()

An `Agent` has attributes:

- Two **types** which can be used to differentiate agents from each other. They can be accessed with `mrAgent.type` and `mrAgent.type2`. In most cases, we only use one type. For example, in a market matching students to schools, we could represent students with `mrAgent.type = "student"`, or simply index it to an integer, as `mrAgent.type = 1`. 
    
    The second type is used to permit more complex relationships between agents. For example, in a kidney transplant market, a transplant list patient is usually accompanied by willing donor, often a family member. However, transplant compatibility depends on blood type, and the patient/donor pair are not necessarily compatible together. We can represent a patient/donor pair as one agent, with each type representing the patient blood type and donor blood type respectively.

In [4]:
PatientDonorPair = mm.Agent()
PatientDonorPair.type = 'AB-'  # Patient blood type
PatientDonorPair.type2 = 'O+'  # Donor blood type

- The **utility of a match** is determined by a dictionary each agent has which maps every other agent to a utility value. This dictionary is accessed by `mrAgent.match_util`.
    This is usually done by type; for example, in a kidney market, we could make the utility $1$ if a match is compatible and $0$ otherwise.
    
- A **discount rate** which is the rate at which the agent loses utility by matching a long time after arriving into the market. This is a floating point number in $[0,1]$. It is often set to $1$; which makes a match have the same value at any time.


- A **match compatibility map**, a dictionary which maps other agents in the market to the probability of a failure of attempting a match with that agent. This dictionary is accessed by `mrAgent.match_fail_prob`. This is often set to $1$ for compatible matches and $0$ for incompatible matches.

    The values in both `match_util` and `match_fail_prob` for all agents are filled by functions which the user can define and which we'll go over later.


- An **Amount of time in the market**, accessed by `mrAgent.time_to_critical`. This is an integer which denotes the number of periods after entering the market before the agent goes critical. When an agent is *critical*, he will leave the market at the next period (for example, death in an organ transplant list). 

    If the market is static (has one period), `time_to_critical` will be $0$ for everyone. Since everyone leaves the market at the next period, they are all critical in the period they enter the market.
    
    
- The `Agent` class also has a few other **automatically generated attributes**:

    - A name, accessed by `mrAgent.name`, which is automatically set to a different integer for all agents in a market. This is to differentiate otherwise similar agents. The integer is indexed to the order in which the agents entered the market.
    
    - A bool, `is_critical`, which is 1 if the agent will perish in the next period and 0 otherwise
    
    - An integer, `sojourn`, which tracks the number of periods the agent has been in the market

# Market

The `Market` class acts as a container for agents, and tracks the interactions between the `Agents` as time advances. It also implements the important `Market.update()` function which implements all the changes in a time period. We'll go over the update function after defining some more machinery.


-  The agents **currently in the market** is a python list accessed by `Market.Agents`

- The **perished** and **matched** agents are similarly pyhon lists, that contain the perished and matched agents, respectively. They are accessed by
 `Market.perished`, and `Market.matched`


   Note we use the term "perished" to denote agents who dropped out of the market without a match after *going critical*. This term is self-explanatory in organ transplant lists, but used in the general case.
    
    
- **Completed matches** are tracked in `Market.matched_dict`, a python dictionary. Note that matches, in the general case, are a represented by a directed graph. In some markets, matches are bilateral, but in others, matching can take the form of a "chain", as in the case of [organ transplants](http://www.bbc.co.uk/news/health-17112314).

- The average amount of **new agents arriving in the market** at any period is kept in `Market.arrival_rate`. It is the $\lambda$ in a poisson distribution, which is randomly drawn from every period.

- The average **probability of a match** between two compatible agents being acceptable is in `Market.acceptable_prob`, a float in $[0,1]$. This can be used to create additional stochastic friction in a market, but is usually set to $1$, nullifying this friction.


- There are other **automatically generated attributes** in the `Market` class:

    - `Market.time`, tracking the age of the market
    
    - `Market.welfare` tracking the total utility of all agents in the market
    
    - `Market.max_agents`, the maximum number of agents that can enter the market over all periods
    
    - `Market.total_agents`, the total distinct number of agents who ever entered market
    
    - `Market.loss`, the rate of perished agents observed in the market up to now

# Matching Algorithms

The matching algorithm implements a matching method **at a particular period**. Matching algorithms take in a python list of `Agents` and a `Market` object, and return a python dictionary, mapping directed matches as `{sourceAgent.name : sinkAgent.name}`. Bilateral matches need to have both agents pointing to each other in the dict, as `{agent1.name: agent2.name, agent2.name: agent1.name}`. 

The user is given flexibility in defining an algorithm, as long as the inputs and outputs are correctly formatted. For instance, the list of `Agents` can represent the pool of agents to be matched by the algorithm, or the pool of agents initiating matches, or receiving matches, etc.

We now have all the machinery necessary to make an algorithm! To make this concrete, let's go over a function implementing random bilateral matches. The agents passed in the function *initiate* matches with anyone in the entire market.

In [11]:
def arbitraryMatch(Mrkt, Agents):
    """
    Random __bilateral__ matches
    Agents in input initiate matches
    Initiating agent can match with anyone
    Arguments
    --------------
    Mrkt: Market object
        The Market in which the matches take place
    Agents: List<Agent object>
        The agents initiating the random bilateral matches
    Returns
    --------------
    dict<Agent.name : Agent.name>
    """
    # matched is the returned dict
    matched = dict()

    # copy list of agents to match to delete on while iterating
    AgentCopy = list(Agents)
    
    allAgentNames = [a.name for a in Mrkt.Agents]
    for agent in Agents:
        # If agent already matched, skip
        if agent not in AgentCopy:
            continue
        myNeighbors = agent.neighbors()
        # remove already matched neighbors
        localAgentNames = [a.name for a in AgentCopy]
        for neighborName in myNeighbors:
            if neighborName in allAgentNames and \
                      neighborName not in localAgentNames:
                myNeighbors.remove(neighborName)
        # If neighbors, match at random
        if len(myNeighbors) > 0:
            match = random.choice(myNeighbors)
            matched[agent.name] = match
            matched[match] = agent.name
            # Clean up matched agents
            AgentCopy.remove(agent)
            for agent_match in AgentCopy:
                if agent_match.name == match:
                    AgentCopy.remove(agent_match)
                    break
    return matched

# Matching Meta-Algorithms

Meta-Algorithms are a wrapper around a matching algorithm that define the **timing and pool of matches** for the matching algorithm. They take in a `Market` object and a matching algorithm as defined above. Again, flexibility is left to users to build timing rules to their heart's content.

Here is a meta-algorithm where the pool of agents is the agents currently *critical* in the market. It implements the random bilateral matching algorithm we defined above by default, so critical agents will *initiate* matches with anyone in the market:

In [12]:
def meta_Patient(Market, match=arbitraryMatch):
    """
    Patient algorithm from Akbarpour et al. (2014)
    Attempts match if agent is critical
    """
    AgentList = [ag for ag in Market.Agents if ag.is_critical]
    return match(Market, AgentList)

Note that we can pass any other matching algorithm into `meta_Patient` as a keyword argument.


# Simulations

To create a simulation, we first initialize a `simulation` object.

In [13]:
tutorialSim = mm.simulation(runs=5, time_per_run=3500, max_agents=5000, logAllData=False)

The arguments creating a `simulation` are attributes for the montecarlo simulation.

- The **number of montecarlo trials** is in `simulation.runs`

- The number of **time periods per run** is in `simulation.time_per_run`

- The **maximum number of agents in a run** is in `simulation.max_agents`

- You can **log all of the data** for every period in every run by setting the `logAllData` flag to true. Note this makes the simulations *much* slower, but this permits you to output plots of your simulations.

The `simulation.run()` method is where most of the action takes place. The method takes many parameters, defining how the market will operate. Here is an example of a call of the function, which we will break down afterwards

In [14]:
from scipy.stats import poisson

tutorialSim.run(arrival_rate=5, time_to_crit=poisson.rvs, crit_input=10,
        algorithm=mm.arbitraryMatch, metaAlgorithm=meta_Patient, 
        matchUtilFct=mm.utilSameType, neighborFct=mm.neighborSameType,
        typeGenerator=mm.randomType, numTypes=5)

tutorialSim.stats()

Simulation Results
3500  periods
5  runs
Stat	  value	 (std dev)
Welfare:   5002.2 ( 2.0396 )
matches:   5002.2  ( 2.0396 )
perished:  0.0  ( 0.0000 )
loss%:     0.0000  ( 0.0000 )


- The **arrival rate**, the **algorithm** and **meta-algorithm** are as we defined them above


- The function that defines **when agents go critical** is in `time_to_crit`. In this example, we took scipy's implementation of a poisson distribution draw. The **input of the function** is in `crit_input`, so here the $\lambda$ in the poisson function is 10


- The **function that generates the types of the agents** is implemented in `typeGenerator`. The `typeGenerator` function takes one input, which is called `numTypes` but can be overloaded with whatever data type the user wants. The function is called every time an agent is created, so it's usually best to make it some form of random draw. 

    Here, `randomType` assigns the `Agent`'s type to be a random integer between $0$ and `numTypes`, so there are $5$ types of agents, with equal probability of each in the market.

- The **function that defines agent compatibility** is called the `neighborFct`, because it returns neighbor vertices in the directed graph of possible matches. To be compatible with the simulation, a `neighborFct` takes two agents as input, and a cutoff value, which is always passed by the `Market.acceptable_prob`. The functions returns an integer, which is the probability of the success of a match. 

  Here, the function passed returns $1$ if the agents have the same type, and $0$ otherwise. Note that we ignore the `cutoff` input, but have to implement it for compatibility:

In [15]:
def neighborSameType(agent, otherAgent, cutoff):
    """
    neighborFct overload
    returns 1 if types are same
    """
    result = agent.type == otherAgent.type \
        and agent.name != otherAgent.name
    return result

- The **function that defines the utility agents receive from a match** is called the `matchUtilFct`. It has *the same formatting as* the `neighborFct`, taking two agents and an integer/floating point number as input, and outputs a number, the utility the source vertex of the match gets from matching with the sink vertex.

    Here, the `utilSameType` function returns $1$ if the match is with an agent of the same type, and $0$ otherwise.

# Using the package

We are now ready to run some simulations!

Let's test an important result from Akbarpour & al. (2014). They prove that, for the arbitrary matching algorithm we implemented above, in bilateral matching, in continuous time, waiting for agents to initiate matches when they are critical (as in the `meta_patient` algorithm we just saw) will outperform a meta-algorithm that matches agents when they enter the market.

In [16]:
def meta_Greedy(Market, match=arbitraryMatch):
    """
    Attempts match if agent is entering market
    """
    AgentList = [ag for ag in Market.Agents if ag.sojourn == 0]
    return match(Market, AgentList)

Additionally, we can test the naive algorithm that always tries to match everyone, every period. Note that this one was not included in Akbarpour & al. (2014).

In [17]:
def meta_Always(Market, match=arbitraryMatch):
    """
    Attempts everyone, every period
    """
    return match(Market, Market.Agents)

Let's compare the performance of these two meta-algorithms under various initial conditions!

**Thin Market**

First, let's try a thin market -- one where few agents have available matches at any time. We can induce this by having a low `arrival_rate`, a low `crit_input`, and a high `numTypes` with the type of market we are studying.

Note that, over time, *market thickness is endogenous on the algorithm and meta-algorithm*. But we are trying to make the market exogenously thinner or thicker to see how our algorithms perform.

In [18]:
thin_market_greedy = mm.simulation(runs=5, time_per_run=35000, max_agents=5000, logAllData=False)
thin_market_patient = mm.simulation(runs=5, time_per_run=35000, max_agents=5000, logAllData=False)
thin_market_always = mm.simulation(runs=5, time_per_run=35000, max_agents=5000, logAllData=False)
    
thin_market_greedy.run(arrival_rate=0.7, time_to_crit=poisson.rvs, crit_input=1.7, 
                       algorithm=mm.arbitraryMatch, metaAlgorithm=mm.meta_Greedy, 
                       matchUtilFct=mm.utilSameType, neighborFct=mm.neighborSameType,
                       typeGenerator=mm.randomType, numTypes=8)
thin_market_patient.run(arrival_rate=0.7, time_to_crit=poisson.rvs, crit_input=1.7, 
                        algorithm=mm.arbitraryMatch, metaAlgorithm=mm.meta_Patient, 
                        matchUtilFct=mm.utilSameType, neighborFct=mm.neighborSameType, 
                        typeGenerator=mm.randomType, numTypes=8)
thin_market_always.run(arrival_rate=0.7, time_to_crit=poisson.rvs, crit_input=1.7, 
                       algorithm=mm.arbitraryMatch, metaAlgorithm=mm.meta_Always, 
                       matchUtilFct=mm.utilSameType, neighborFct=mm.neighborSameType,
                       typeGenerator=mm.randomType, numTypes=8)


print("GREEDY ALGORITHM RESULTS\n---Thin Market---")

thin_market_greedy.stats()

print("\n\nPATIENT ALGORITHM RESULTS\n---Thin Market---")

thin_market_patient.stats()

print("\n\nALWAYS-MATCH ALGORITHM RESULTS\n---Thin Market---")

thin_market_always.stats()

GREEDY ALGORITHM RESULTS
---Thin Market---
Simulation Results
35000  periods
5  runs
Stat	  value	 (std dev)
Welfare:   485.6 ( 10.3653 )
matches:   485.6  ( 10.3653 )
perished:  4514.6  ( 10.1509 )
loss%:     0.9029  ( 0.0021 )


PATIENT ALGORITHM RESULTS
---Thin Market---
Simulation Results
35000  periods
5  runs
Stat	  value	 (std dev)
Welfare:   1059.4 ( 24.8564 )
matches:   1059.4  ( 24.8564 )
perished:  3941.0  ( 25.1157 )
loss%:     0.7881  ( 0.0050 )


ALWAYS-MATCH ALGORITHM RESULTS
---Thin Market---
Simulation Results
35000  periods
5  runs
Stat	  value	 (std dev)
Welfare:   1462.6 ( 39.0210 )
matches:   1462.6  ( 39.0210 )
perished:  3537.6  ( 39.0518 )
loss%:     0.7075  ( 0.0078 )


As we can see, in a thin market, the greedy algorithm performs poorly, the patient algorithm performs better, and trying to always match agents does best of the three.

Note that here welfare is the same thing as the number of matches, because utility of matching succesfully is 1, and probability of failure of a match is 0.

**Thick Market**

Let's now try a thick market by adjusting the input parameters.

In [26]:
thick_market_greedy = mm.simulation(runs=5, time_per_run=3500, max_agents=5000, logAllData=False)
thick_market_patient = mm.simulation(runs=5, time_per_run=3500, max_agents=5000, logAllData=False)
thick_market_always = mm.simulation(runs=5, time_per_run=3500, max_agents=5000, logAllData=False)
    
thick_market_greedy.run(arrival_rate=3, time_to_crit=poisson.rvs, crit_input=8, 
                        algorithm=mm.arbitraryMatch, metaAlgorithm=mm.meta_Greedy, 
                        matchUtilFct=mm.utilSameType, neighborFct=mm.neighborSameType,
                        typeGenerator=mm.randomType, numTypes=4)
thick_market_patient.run(arrival_rate=3, time_to_crit=poisson.rvs, crit_input=8, 
                         algorithm=mm.arbitraryMatch, metaAlgorithm=mm.meta_Patient, 
                         matchUtilFct=mm.utilSameType, neighborFct=mm.neighborSameType,
                         typeGenerator=mm.randomType, numTypes=4)
thick_market_always.run(arrival_rate=3, time_to_crit=poisson.rvs, crit_input=8, 
                        algorithm=mm.arbitraryMatch, metaAlgorithm=mm.meta_Always, 
                        matchUtilFct=mm.utilSameType, neighborFct=mm.neighborSameType,
                        typeGenerator=mm.randomType, numTypes=4)


print("GREEDY ALGORITHM RESULTS\n---Thick Market---")

thick_market_greedy.stats()

print("\n\nPATIENT ALGORITHM RESULTS\n---Thick Market---")

thick_market_patient.stats()

print("\n\nALWAYS-MATCH ALGORITHM RESULTS\n---Thick Market---")

thick_market_always.stats()

GREEDY ALGORITHM RESULTS
---Thick Market---
Simulation Results
3500  periods
5  runs
Stat	  value	 (std dev)
Welfare:   4371.6 ( 22.0690 )
matches:   4371.6  ( 22.0690 )
perished:  629.2  ( 21.2546 )
loss%:     0.1258  ( 0.0043 )


PATIENT ALGORITHM RESULTS
---Thick Market---
Simulation Results
3500  periods
5  runs
Stat	  value	 (std dev)
Welfare:   4999.8 ( 3.1241 )
matches:   4999.8  ( 3.1241 )
perished:  2.2  ( 1.4697 )
loss%:     0.0004  ( 0.0003 )


ALWAYS-MATCH ALGORITHM RESULTS
---Thick Market---
Simulation Results
3500  periods
5  runs
Stat	  value	 (std dev)
Welfare:   4977.8 ( 5.0359 )
matches:   4977.8  ( 5.0359 )
perished:  23.8  ( 4.5782 )
loss%:     0.0048  ( 0.0009 )


We already have an interesting result! **In a thicker market, the patient algorithm does even better than the always-match algorithm!** We can also see that the greedy algorithm continues to perform poorly.

While we haven't formally proved any results, we can perform these montecarlo trials quickly over a larger input space and get a good idea of algorithm performance.

In part 2, we'll study more sophisticated matching algorithms, and see how they pair with meta-algorithms.